In [32]:
# import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
import xgboost
from sklearn import svm,tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score


import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from tensorflow.keras.layers import Embedding,Input,LSTM,Dense,Bidirectional,Dropout, Activation
from keras.models import Model
from tensorflow.keras.models import Sequential
tf.__version__
import warnings
warnings.filterwarnings("ignore")

In [33]:
# Load the data
df = pd.read_csv('input/emails.csv')
#df.head()

In [34]:
# dropping the duplicate columns
df.drop_duplicates(inplace = True)

Number of duplicate rows in the data are :  33 
So we drop them


In [57]:
# Checking counts of spams and non-spams
df['spam'].value_counts()

0    4327
1    1368
Name: spam, dtype: int64

## Preprocessing data

In [48]:
#1.Punctuations are [!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]
#2.Stop words in natural language processing, are useless words (data).
def text_process(text):
  STOPWORDS = set(stopwords.words('english'))
  STOPWORDS.add('subject')
  
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = [word for word in text.split() if word.lower() not in STOPWORDS]
  return " ".join(text)

In [49]:
#Show the processed data
df.text = df.text.apply(text_process)
df.text.head()

0    naturally irresistible corporate identity lt r...
1    stock trading gunslinger fanny merrill muzo co...
2    unbelievable new homes made easy im wanting sh...
3    4 color printing special request additional in...
4    money get software cds software compatibility ...
Name: text, dtype: object

## Vectorization of the text data

In [50]:
vocab_size = 10000
max_len = 250

# Tokenize the mails
tok = Tokenizer(num_words=vocab_size)
tok.fit_on_texts(df.text)

# Use text_to_sequence to convert it into vectors
sequences = tok.texts_to_sequences(df.text)

# pad seqence to create a matrix of equal length mails
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [51]:
sequences_matrix[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 5664,  408, 1651, 3769,
        352,  783, 4270,   49,   58,  296, 6153,   

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sequences_matrix, df.spam, test_size = 0.2, random_state = 1)

In [53]:
model = Sequential()
model.add(Embedding(vocab_size, 200, input_length=max_len))
model.add(LSTM(32))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 200)          2000000   
                                                                 
 lstm_1 (LSTM)               (None, 32)                29824     
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,029,857
Trainable params: 2,029,857
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Epoch 1/5
72/72 [==============================] - 27s 297ms/step - loss: 0.3058 - accuracy: 0.8837 - val_loss: 0.1009 - val_accuracy: 0.9816
Epoch 2/5
72/72 [==============================] - 20s 276ms/step - loss: 0.0506 - accuracy: 0.9928 - val_loss: 0.0500 - val_accuracy: 0.9886
Epoch 3/5
72/72 [==============================] - 20s 285ms/step - loss: 0.0135 - accuracy: 0.9991 - val_loss: 0.0431 - val_accuracy: 0.9868
Epoch 4/5
72/72 [==============================] - 19s 268ms/step - loss: 0.0175 - accuracy: 0.9965 - val_loss: 0.0848 - val_accuracy: 0.9701
Epoch 5/5
72/72 [==============================] - 20s 276ms/step - loss: 0.0110 - accuracy: 0.9987 - val_loss: 0.0636 - val_accuracy: 0.9816


In [55]:
scores = model.evaluate(X_test, y_test, verbose=0)
predict_x=model.predict(X_test) 
y_pred=np.argmax(predict_x,axis=1)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('confusion matrix:\n', confusion_matrix(y_pred,y_test))

36/36 [==============================] - 3s 57ms/step
Test loss: 0.06355982273817062
Test accuracy: 0.9815627932548523
confusion matrix:
 [[866 273]
 [  0   0]]
